# Data Collection Pipeline V2 - Pareto-Based Evaluation

This notebook collects 200 training samples using a **better approach**:

## Old Approach (V1):
1. Sample random preferences
2. Use fixed heuristic to translate preferences → costs
3. Optimize with those costs
4. Save (preferences, costs, features)

**Problem**: Model just learns our fixed heuristic!

## New Approach (V2):
1. Sample random preferences
2. Generate 5 random cost sets **independently**
3. Optimize each cost set (5 solutions)
4. Score each solution against preferences
5. Pick the cost set that produces the best-matching solution
6. Save (preferences, best_costs, best_features)

**Benefit**: Model learns the actual relationship between preferences and optimal costs!

**Estimated time**: ~15-20 hours (200 samples × 5 optimizations × 90 seconds each)

In [ ]:
import json
import time
import requests
import numpy as np
import pandas as pd
from datetime import datetime
from pathlib import Path
import copy

print("✓ Imports successful")

## Configuration

**⚠️ CHANGE THESE VALUES:**

In [ ]:
# API Configuration


# Model file (25 jobs, 8 vehicles)
MODEL_PATH = "test_model_25jobs_8vehicles.json"  # ← Your model file

# Data collection settings
N_SAMPLES = 200
N_COST_CANDIDATES = 5  # Number of random cost sets to try per sample
OUTPUT_FILE = "training_data_v2.json"
CHECKPOINT_EVERY = 5  # Save every 5 samples (since each takes longer)
WAIT_SECONDS = 90  # Wait time for each optimization

# Cost parameter ranges
COST_RANGES = {
    'costPerTravelHour': (0.5, 5.0),
    'costPerKm': (0.014, 0.49),
    'parking_multiplier': (0.2, 5.0)
}

print(f"Configuration:")
print(f"  Base URL: {BASE_URL}")
print(f"  Model: {MODEL_PATH}")
print(f"  Samples to collect: {N_SAMPLES}")
print(f"  Cost candidates per sample: {N_COST_CANDIDATES}")
print(f"  Output: {OUTPUT_FILE}")
print(f"  Estimated time: {(N_SAMPLES * N_COST_CANDIDATES * WAIT_SECONDS) / 3600:.1f} hours")

## Data Collector Class with Pareto Evaluation

In [ ]:
class ParetoDataCollector:
    """Collects training data using Pareto-based evaluation"""
    
    def __init__(self, base_url, username, password, model_path, cost_ranges):
        """Initialize data collector"""
        self.base_url = base_url.rstrip('/')
        self.username = username
        self.password = password
        self.cost_ranges = cost_ranges
        
        # Initialize session
        self.session = requests.Session()
        self.session.auth = (username, password)
        self.session.headers.update({
            'Content-Type': 'application/json',
            'Accept': 'application/json'
        })
        
        # Load base model
        with open(model_path, 'r') as f:
            self.base_model = json.load(f)
        
        print(f"✓ Base model loaded")
        print(f"  Jobs: {len(self.base_model['data']['jobs'])}")
        print(f"  Vehicles: {len(self.base_model['data']['vehicles'])}")
        
        # Data storage
        self.collected_data = []
    
    def sample_preferences(self):
        """Sample random preference values"""
        preferences = {
            'parking_importance': np.random.uniform(0.0, 1.0),
            'time_importance': np.random.uniform(0.0, 1.0),
            'distance_importance': np.random.uniform(0.0, 1.0)
        }
        return preferences
    
    def sample_random_costs(self):
        """Sample random cost parameters from valid ranges"""
        costs = {}
        for param, (min_val, max_val) in self.cost_ranges.items():
            costs[param] = np.random.uniform(min_val, max_val)
        
        # Round for cleaner values
        costs['costPerTravelHour'] = round(costs['costPerTravelHour'], 2)
        costs['costPerKm'] = round(costs['costPerKm'], 4)
        costs['parking_multiplier'] = round(costs['parking_multiplier'], 2)
        
        return costs
    
    def apply_costs_to_model(self, costs):
        """Apply cost parameters to model"""
        model = copy.deepcopy(self.base_model)
        
        # Apply vehicle costs
        for vehicle in model['data']['vehicles']:
            if 'definition' in vehicle:
                vehicle['definition']['costPerTravelHour'] = costs['costPerTravelHour']
                vehicle['definition']['costPerKm'] = costs['costPerKm']
        
        # Apply parking multiplier
        for job in model['data']['jobs']:
            for stop in job.get('stops', []):
                if 'parking' in stop and 'cost' in stop['parking']:
                    original_cost = stop['parking']['cost']
                    stop['parking']['cost'] = original_cost * costs['parking_multiplier']
        
        return model
    
    def send_to_odl(self, model, model_id):
        """Send model to ODL API"""
        url = f"{self.base_url}/models/{model_id}"
        
        try:
            response = self.session.put(url, json=model, timeout=30)
            response.raise_for_status()
            return True
        except Exception as e:
            print(f"  ✗ Upload failed: {e}")
            return False
    
    def get_plan(self, model_id):
        """Retrieve plan from ODL API"""
        url = f"{self.base_url}/models/{model_id}/plan"
        
        try:
            response = self.session.get(url, timeout=30)
            response.raise_for_status()
            return response.json()
        except Exception as e:
            print(f"  ✗ Get plan failed: {e}")
            return None
    
    def extract_features(self, plan):
        """Extract route features from plan"""
        if not plan or 'vehiclePlans' not in plan:
            return None
        
        features = {
            'total_distance_km': 0.0,
            'total_travel_hours': 0.0,
            'total_cost': 0.0,
            'num_stops': 0,
            'num_vehicles_used': 0,
            'unplanned_jobs': len(plan.get('unplannedJobs', [])),
            'avg_parking_difficulty': 0.0
        }
        
        total_parking_cost = 0.0
        parking_stops = 0
        
        for vehicle_plan in plan['vehiclePlans']:
            stops = vehicle_plan.get('plannedStops', [])
            if len(stops) > 0:
                features['num_vehicles_used'] += 1
            
            for stop in stops:
                features['num_stops'] += 1
                
                # Accumulate metrics
                features['total_distance_km'] += stop.get('travelDistanceFromPrevious', 0.0)
                features['total_travel_hours'] += stop.get('travelDurationFromPrevious', 0.0) / 3600.0
                
                # Parking cost
                parking = stop.get('parking', {})
                if 'cost' in parking:
                    total_parking_cost += parking['cost']
                    parking_stops += 1
        
        # Calculate averages
        if parking_stops > 0:
            features['avg_parking_difficulty'] = total_parking_cost / parking_stops
        
        # Get total cost from statistics
        if 'statistics' in plan and 'totalCost' in plan['statistics']:
            features['total_cost'] = plan['statistics']['totalCost']
        
        # Round for cleaner values
        features['total_distance_km'] = round(features['total_distance_km'], 2)
        features['total_travel_hours'] = round(features['total_travel_hours'], 2)
        features['total_cost'] = round(features['total_cost'], 2)
        features['avg_parking_difficulty'] = round(features['avg_parking_difficulty'], 2)
        
        return features
    
    def score_solution(self, preferences, features):
        """
        Score how well a solution matches preferences.
        Lower score = better match
        
        This is the key Pareto evaluation function!
        """
        if features is None:
            return float('inf')  # Invalid solution gets worst score
        
        score = 0.0
        
        # Parking importance: weight the parking difficulty
        score += preferences['parking_importance'] * features['avg_parking_difficulty']
        
        # Time importance: weight the travel hours
        score += preferences['time_importance'] * features['total_travel_hours'] * 10  # Scale up for balance
        
        # Distance importance: weight the distance
        score += preferences['distance_importance'] * features['total_distance_km'] * 0.1  # Scale down for balance
        
        # Penalty for unplanned jobs (these are always bad)
        score += features['unplanned_jobs'] * 1000
        
        return score
    
    def collect_sample_with_evaluation(self, sample_num, n_candidates, wait_seconds):
        """
        Collect one training sample with Pareto evaluation:
        1. Sample preferences
        2. Try N random cost sets
        3. Pick the one that best matches preferences
        """
        print(f"\n{'='*60}")
        print(f"Sample {sample_num}: Starting Pareto evaluation")
        print(f"{'='*60}")
        
        # 1. Sample preferences
        preferences = self.sample_preferences()
        print(f"\nPreferences:")
        print(f"  Parking importance: {preferences['parking_importance']:.2f}")
        print(f"  Time importance: {preferences['time_importance']:.2f}")
        print(f"  Distance importance: {preferences['distance_importance']:.2f}")
        
        # 2. Try multiple cost candidates
        candidates = []
        
        for i in range(n_candidates):
            print(f"\n  Candidate {i+1}/{n_candidates}:")
            
            # Generate random costs
            costs = self.sample_random_costs()
            print(f"    Costs: hourly={costs['costPerTravelHour']:.2f}, "
                  f"km={costs['costPerKm']:.4f}, parking_mult={costs['parking_multiplier']:.2f}")
            
            # Apply to model and send to API
            model = self.apply_costs_to_model(costs)
            model_id = f"training_sample_{sample_num}_candidate_{i}"
            
            if not self.send_to_odl(model, model_id):
                print(f"    ✗ Failed to send model")
                continue
            
            print(f"    ✓ Model uploaded, waiting {wait_seconds}s for optimization...")
            time.sleep(wait_seconds)
            
            # Get results
            plan = self.get_plan(model_id)
            features = self.extract_features(plan)
            
            if features is None:
                print(f"    ✗ Failed to extract features")
                continue
            
            # Score this solution
            score = self.score_solution(preferences, features)
            
            print(f"    Features: dist={features['total_distance_km']:.1f}km, "
                  f"time={features['total_travel_hours']:.1f}h, "
                  f"parking={features['avg_parking_difficulty']:.1f}")
            print(f"    Score: {score:.2f} (lower is better)")
            
            candidates.append({
                'costs': costs,
                'features': features,
                'score': score
            })
        
        # 3. Find best candidate
        if len(candidates) == 0:
            print(f"\n✗ No valid candidates for sample {sample_num}")
            return None
        
        best_candidate = min(candidates, key=lambda x: x['score'])
        
        print(f"\n{'='*60}")
        print(f"BEST CANDIDATE (score: {best_candidate['score']:.2f}):")
        print(f"  Costs: {best_candidate['costs']}")
        print(f"  Features: {best_candidate['features']}")
        print(f"{'='*60}")
        
        # Create training sample
        sample = {
            'sample_num': sample_num,
            'preferences': preferences,
            'costs': best_candidate['costs'],
            'features': best_candidate['features'],
            'score': best_candidate['score'],
            'num_candidates_tried': len(candidates),
            'timestamp': datetime.now().isoformat()
        }
        
        return sample
    
    def save_data(self, output_file):
        """Save collected data to JSON file"""
        with open(output_file, 'w') as f:
            json.dump(self.collected_data, f, indent=2)
        print(f"\n💾 Saved {len(self.collected_data)} samples to {output_file}")

## Initialize Collector

In [ ]:
# Validate configuration
if USERNAME == "your-username" or PASSWORD == "your-password":
    print("❌ ERROR: Please set your USERNAME and PASSWORD in the configuration cell above!")
else:
    # Initialize collector
    collector = ParetoDataCollector(
        base_url=BASE_URL,
        username=USERNAME,
        password=PASSWORD,
        model_path=MODEL_PATH,
        cost_ranges=COST_RANGES
    )
    print("\n✓ Pareto collector initialized and ready!")

## Test with Single Sample

Run this first to make sure everything works. This will:
- Sample 1 set of preferences
- Try 5 different cost sets
- Pick the best one

**This will take ~7-8 minutes** (5 optimizations × 90 seconds)

In [ ]:
# Test with one sample
test_sample = collector.collect_sample_with_evaluation(
    sample_num=0,
    n_candidates=N_COST_CANDIDATES,
    wait_seconds=WAIT_SECONDS
)

if test_sample:
    print("\n✓ Test successful!")
    print("\nSample data structure:")
    print(json.dumps(test_sample, indent=2))
else:
    print("\n✗ Test failed - check your configuration")

## Collect Full Dataset

⚠️ **This will take ~15-20 hours to complete!**

The notebook will:
- Collect 200 samples
- Each sample tries 5 cost candidates (Pareto evaluation)
- Save checkpoints every 5 samples
- Show detailed progress

You can interrupt with Kernel → Interrupt and your data will be saved.

In [ ]:
print(f"\n{'='*60}")
print(f"DATA COLLECTION PIPELINE V2 (PARETO)")
print(f"{'='*60}")
print(f"Target samples: {N_SAMPLES}")
print(f"Cost candidates per sample: {N_COST_CANDIDATES}")
print(f"Total optimizations: {N_SAMPLES * N_COST_CANDIDATES}")
print(f"Output file: {OUTPUT_FILE}")
print(f"Checkpoint every: {CHECKPOINT_EVERY} samples")
print(f"Wait time: {WAIT_SECONDS} seconds per optimization")
print(f"Estimated total time: {(N_SAMPLES * N_COST_CANDIDATES * WAIT_SECONDS) / 3600:.1f} hours")
print(f"{'='*60}\n")

start_time = time.time()

try:
    for i in range(1, N_SAMPLES + 1):
        try:
            sample = collector.collect_sample_with_evaluation(
                sample_num=i,
                n_candidates=N_COST_CANDIDATES,
                wait_seconds=WAIT_SECONDS
            )
            
            if sample is not None:
                collector.collected_data.append(sample)
                print(f"\n✓ Sample {i} collected successfully")
            else:
                print(f"\n✗ Sample {i} failed, skipping...")
            
            # Save checkpoint
            if i % CHECKPOINT_EVERY == 0:
                collector.save_data(OUTPUT_FILE)
                elapsed = time.time() - start_time
                avg_time = elapsed / i
                remaining = (N_SAMPLES - i) * avg_time
                print(f"\n{'='*60}")
                print(f"CHECKPOINT: {i}/{N_SAMPLES} samples collected")
                print(f"Elapsed: {elapsed/3600:.1f} hours")
                print(f"Estimated remaining: {remaining/3600:.1f} hours")
                print(f"Average time per sample: {avg_time/60:.1f} minutes")
                print(f"{'='*60}\n")
                
        except Exception as e:
            print(f"\n✗ Unexpected error on sample {i}: {e}")
            continue
            
except KeyboardInterrupt:
    print(f"\n\nInterrupted by user!")
    print(f"Collected {len(collector.collected_data)} samples so far")

# Final save
collector.save_data(OUTPUT_FILE)

elapsed = time.time() - start_time
print(f"\n{'='*60}")
print(f"DATA COLLECTION COMPLETE")
print(f"{'='*60}")
print(f"Total samples collected: {len(collector.collected_data)}")
print(f"Total time: {elapsed/3600:.1f} hours")
if len(collector.collected_data) > 0:
    print(f"Average time per sample: {(elapsed/len(collector.collected_data))/60:.1f} minutes")
    print(f"Total optimizations run: {len(collector.collected_data) * N_COST_CANDIDATES}")
print(f"Data saved to: {OUTPUT_FILE}")
print(f"{'='*60}\n")

## Analyze Collected Data

In [ ]:
# Load the collected data
with open(OUTPUT_FILE, 'r') as f:
    data = json.load(f)

print(f"Dataset Summary:")
print(f"  Total samples: {len(data)}")

# Convert to DataFrame for analysis
df_records = []
for sample in data:
    record = {
        'sample_num': sample['sample_num'],
        **{f'pref_{k}': v for k, v in sample['preferences'].items()},
        **{f'cost_{k}': v for k, v in sample['costs'].items()},
        **{f'feat_{k}': v for k, v in sample['features'].items()},
        'score': sample['score'],
        'num_candidates_tried': sample['num_candidates_tried']
    }
    df_records.append(record)

df = pd.DataFrame(df_records)

print(f"\nDataFrame shape: {df.shape}")
print(f"\nFirst few rows:")
print(df.head())

print(f"\nPreferences statistics:")
print(df[['pref_parking_importance', 'pref_time_importance', 'pref_distance_importance']].describe())

print(f"\nCosts statistics (these are the LEARNED optimal costs):")
print(df[['cost_costPerTravelHour', 'cost_costPerKm', 'cost_parking_multiplier']].describe())

print(f"\nFeatures statistics:")
print(df[['feat_total_distance_km', 'feat_total_travel_hours', 'feat_avg_parking_difficulty']].describe())

print(f"\nScore statistics (lower is better):")
print(df['score'].describe())

## Check Data Quality

Let's verify that the Pareto evaluation is working correctly.

In [ ]:
# Example: Look at a few samples where parking importance is high
print("Samples with HIGH parking importance (>0.8):")
high_parking = df[df['pref_parking_importance'] > 0.8].head(3)
print(high_parking[['pref_parking_importance', 'cost_parking_multiplier', 
                     'feat_avg_parking_difficulty', 'score']])

print("\nSamples with LOW parking importance (<0.2):")
low_parking = df[df['pref_parking_importance'] < 0.2].head(3)
print(low_parking[['pref_parking_importance', 'cost_parking_multiplier', 
                    'feat_avg_parking_difficulty', 'score']])

print("\nExpected pattern: High parking importance should correlate with high parking_multiplier")
print(f"Correlation: {df['pref_parking_importance'].corr(df['cost_parking_multiplier']):.3f}")

## Next Steps

Now that you have collected training data with Pareto evaluation:

1. ✅ **Data collected**: `training_data_v2.json`
2. ✅ **Learned from actual optimization results** (not a fixed heuristic!)
3. 🔜 **Train regression model**: preferences → costs
4. 🔜 **Validate model performance**
5. 🔜 **Compare V1 vs V2 model accuracy**

The V2 model should generalize better because it learned the true relationship!